# Week 2

## Monday May 28, 2019:
* created new SURF_2019 branch of FindMyFriends starting at commit 01753b8fc715284fd8e4fa30621ef5e2c78891e3 (March 3, 2016, Version Bump)
* read through aaa.R and other files to see how pre-CD-Hit FindMyFriends clustered data
* read some methods from [kebab R package](https://www.bioconductor.org/packages/devel/bioc/manuals/kebabs/man/kebabs.pdf) 

I read through the vignette for the package and learned about the main features of FindMyFriends (FMF). FMF reads .fasta (or .faa) files into a pangenome object, and then the pangenome object is manipulated and clustered to determine the pan and core genomes for the input files. 

The first step in FMF is to cluster the genomes based on kmer counts. This is done using a technique called guided pairwise comparison, where a tree is constructed and grouped to represent the relationships between the input sequences. To construct the tree, the genes for each genome are concatenated, kmer count profiles are created for each genome, and then genomes are compared based on their overall kmer count profiles (by calculating the normalized inner product of each vector against each other vector, spectrumkernel from kebab). This similarity matrix is then converted to a distance matrix using the euclidean metric between each genome's kmer count profile, and the distance matrix is clustered (using hclust, method = ward.D2) to generate a dendrogram. Subtrees of the dendrogram are then compared and grouped, and each time a subtree is grouped with another subtree, a random sample is chosen to represent the subtree for the next comparison. This tree can then be further analyzed. 

After reading and learning about FindMyFriends, I installed the package (along with a ton of dependencies!) and tested out the functionality using the examples from the vignette. I used 9 Mycoplasma pneumonia genomes that were provided in the package. 

I was able to group the genes using guided pairwise comparison (gpc) and classify the genes into core, accessory, or singleton categories: ![geneGrouping](images/w2/geneGrouping.png)
I could also get broad statistics about the genomes: ![stats](images/w2/broadStats.png) ![summaryGraphs](images/w2/summaryGraphs.png) ![evo](images/w2/plotEvolution.png)

I was also able to generate two different heatmaps for the genomes. The first groups the genomes based on the percentage of shared genes: ![sharedGenesHeatmap](images/w2/percentSharedGenesHeatmap.png)
I also generated a heatmap of the genomes using cosine similarity distances of the kmer feature vectors for each genome, and it was very similar to the previous heatmaps with the same genome groupings: ![kmerHeatmap](images/w2/5merSimHeatmap.png)

For fun, I also generated a circular phylogenetic tree... ![circularTree](images/w2/circularTree.png)
and a panchromosome map that tries to link the genes based on similar neighboring genes: ![panc](images/w2/panchromosome.png) although I'm not too sure what information (if any) I could actually get from this graph. 

Next, I decided to try to run FindMyFriends for the 26 genomes that were analyzed last week. Code:



In [ ]:
library(FindMyFriends)
library(igraph)

setwd("/Users/matthewthompson/Documents/UAMS_SURF/K-mer_testing/FAA_files")
genomeFiles <- list.files(getwd(), full.names=TRUE, pattern='*.faa')
pan <- pangenome(genomeFiles[1:27], translated=TRUE, geneLocation='prodigal', lowMem=FALSE)
pan <- gpcGrouping(pan, lowerLimit=0.5)
pan

# pie chart and gene graph
plotStat(mycoPan, type='qual', palette=6)

# see how pangenome composition changes as genomes are added
plotEvolution(mycoPan)

# heatmap based on % of similar genes
plotSimilarity(mycoPan)

# heatmap based on kmer similarity
plotSimilarity(mycoPan, type='kmer', kmerSize=5)

Unfortunately, RStudio crashed while running the gpcGrouping analysis for all 27 genomes. I decided to try to run it with a smaller set of genomes. It took around 19 minutes for the gpcGrouping to finish for the 10 genomes. Results:
![summPlot](images/w2/10sumplot.png)
![evoPlot](images/w2/10evoplot.png)
Clustered based on % gene similarity
![tree](images/w2/phylotree.png)
Split by % genes shared:
![pHM](images/w2/10percentHM.png)
Split by kmer profiles:
![kHM](images/w2/10kmerHM.png)

To check and see if FMF is reproducible, I reran the analysis and got slightly different gene classification ratios. 
![summPlot](images/w2/Re10SummPlot.png)
Clustered based on % gene similarity
![tree](images/w2/ReTree.png)
Split by % genes shared:
![pHM](images/w2/RePHM.png)
Split by kmer profiles:
![kHM](images/w2/ReKmerHM.png)

The heatmaps and tree still look the same, but I will have to continue investigating tomorrow. 
